In [ ]:
print('train data shape :', train.shape)
print('test data shape :', test.shape)

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2023/train_data.csv
/kaggle/input/widsdatathon2023/test_data.csv
/kaggle/input/widsdatathon2023/sample_solution.csv


  ## IMPORTING LIBRARIES

In [3]:
# dataframe and plotting
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# machine learning
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

## LOAD DATASETS FROM KAGGLE

In [4]:
train = pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv')
test = pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv')
ss = pd.read_csv('/kaggle/input/widsdatathon2023/sample_solution.csv')

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375734 entries, 0 to 375733
Columns: 246 entries, index to wind-vwnd-925-2010-20
dtypes: float64(240), int64(4), object(2)
memory usage: 705.2+ MB


## EDA

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31354 entries, 0 to 31353
Columns: 245 entries, index to wind-vwnd-925-2010-20
dtypes: float64(241), int64(2), object(2)
memory usage: 58.6+ MB


In [ ]:
ss.head()

COMBINE ALL DATA FOR EASY PRE PROCESSING

In [8]:
# Getting the last index of training data
ntrain = train.shape[0]

# Combining the 2 Dataframes
all_data = pd.concat([train, test]).reset_index(drop=True)

In [9]:
len(all_data.index) 

407088

In [10]:
all_data.describe()

,index,lat,lon,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,...,wind-vwnd-925-2010-11,wind-vwnd-925-2010-12,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20
count,407088.000000,407088.000000,407088.000000,407088.000000,407088.000000,407088.000000,391154.000000,407088.000000,407088.000000,407088.000000,...,407088.000000,407088.000000,407088.000000,407088.000000,407088.000000,407088.000000,407088.000000,407088.000000,407088.000000,407088.000000
mean,203543.500000,0.592766,0.517964,278.477044,11.461094,13.379028,10.829164,12.293069,11.313677,12.436866,...,0.580051,-10.133169,8.239823,20.612247,-17.974280,-8.283699,-6.728258,15.413422,8.797967,12.050164
std,117516.327529,0.251744,0.272059,193.148785,10.956420,11.055192,10.418488,10.548619,9.381164,10.706542,...,37.380863,36.122262,38.244880,25.197315,31.326227,33.350366,27.295851,26.264174,25.322790,24.713421
min,0.000000,0.000000,0.000000,-2.310000,-14.960000,-14.970000,-11.720000,-17.560000,-13.720000,-13.330000,...,-94.570000,-89.940000,-64.870000,-39.280000,-100.560000,-106.080000,-93.980000,-52.610000,-66.110000,-57.610000
25%,101771.750000,0.409091,0.300000,113.190000,2.230000,4.340000,2.280000,3.850000,4.020000,3.700000,...,-25.312500,-35.085000,-18.780000,3.255000,-40.030000,-29.585000,-21.982500,-1.395000,-10.037500,-4.487500
50%,203543.500000,0.590909,0.533333,256.145000,11.130000,13.020000,10.700000,12.690000,11.860000,12.390000,...,-0.690000,-11.490000,3.555000,21.155000,-16.815000,-5.430000,-6.480000,18.305000,9.860000,12.675000
75%,305315.250000,0.818182,0.766667,413.930000,20.900000,22.650000,19.440000,20.930000,18.800000,21.010000,...,25.420000,16.610000,30.715000,36.572500,4.007500,12.092500,11.785000,30.907500,27.422500,29.612500
max,407087.000000,1.000000,1.000000,1185.140000,36.080000,36.030000,33.390000,34.640000,35.750000,38.350000,...,116.770000,96.550000,106.420000,91.760000,56.550000,83.890000,64.850000,90.390000,76.890000,88.660000


shrink the size to reduce memory usage

In [11]:
#reduce the size of the memory usage by the training set
from fastai.tabular.core import df_shrink
all_data= df_shrink(train)
print(all_data.memory_usage().sum()/2**20)

347.60015869140625


In [12]:
pd.set_option('display.max_columns', None) #to display all the columns
all_data.head()


,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,nmme0-tmp2m-34w__gfdlflorb0,nmme0-tmp2m-34w__gfdl0,nmme0-tmp2m-34w__nasa0,nmme0-tmp2m-34w__nmme0mean,contest-wind-h10-14d__wind-hgt-10,nmme-tmp2m-56w__cancm3,nmme-tmp2m-56w__cancm4,nmme-tmp2m-56w__ccsm3,nmme-tmp2m-56w__ccsm4,nmme-tmp2m-56w__cfsv2,nmme-tmp2m-56w__gfdl,nmme-tmp2m-56w__gfdlflora,nmme-tmp2m-56w__gfdlflorb,nmme-tmp2m-56w__nasa,nmme-tmp2m-56w__nmmemean,contest-rhum-sig995-14d__rhum,nmme-prate-34w__cancm3,nmme-prate-34w__cancm4,nmme-prate-34w__ccsm3,nmme-prate-34w__ccsm4,nmme-prate-34w__cfsv2,nmme-prate-34w__gfdl,nmme-prate-34w__gfdlflora,nmme-prate-34w__gfdlflorb,nmme-prate-34w__nasa,nmme-prate-34w__nmmemean,contest-wind-h100-14d__wind-hgt-100,nmme0-prate-56w__cancm30,nmme0-prate-56w__cancm40,nmme0-prate-56w__ccsm30,nmme0-prate-56w__ccsm40,nmme0-prate-56w__cfsv20,nmme0-prate-56w__gfdlflora0,nmme0-prate-56w__gfdlflorb0,nmme0-prate-56w__gfdl0,nmme0-prate-56w__nasa0,nmme0-prate-56w__nmme0mean,nmme0-prate-34w__cancm30,nmme0-prate-34w__cancm40,nmme0-prate-34w__ccsm30,nmme0-prate-34w__ccsm40,nmme0-prate-34w__cfsv20,nmme0-prate-34w__gfdlflora0,nmme0-prate-34w__gfdlflorb0,nmme0-prate-34w__gfdl0,nmme0-prate-34w__nasa0,nmme0-prate-34w__nmme0mean,contest-tmp2m-14d__tmp2m,contest-slp-14d__slp,contest-wind-vwnd-925-14d__wind-vwnd-925,nmme-prate-56w__cancm3,nmme-prate-56w__cancm4,nmme-prate-56w__ccsm3,nmme-prate-56w__ccsm4,nmme-prate-56w__cfsv2,nmme-prate-56w__gfdl,nmme-prate-56w__gfdlflora,nmme-prate-56w__gfdlflorb,nmme-prate-56w__nasa,nmme-prate-56w__nmmemean,contest-pres-sfc-gauss-14d__pres,contest-wind-uwnd-250-14d__wind-uwnd-250,nmme-tmp2m-34w__cancm3,nmme-tmp2m-34w__cancm4,nmme-tmp2m-34w__ccsm3,nmme-tmp2m-34w__ccsm4,nmme-tmp2m-34w__cfsv2,nmme-tmp2m-34w__gfdl,nmme-tmp2m-34w__gfdlflora,nmme-tmp2m-34w__gfdlflorb,nmme-tmp2m-34w__nasa,nmme-tmp2m-34w__nmmemean,contest-prwtr-eatm-14d__prwtr,contest-wind-vwnd-250-14d__wind-vwnd-250,contest-precip-14d__precip,contest-wind-h850-14d__wind-hgt-850,contest-wind-uwnd-925-14d__wind-uwnd-925,contest-wind-h500-14d__wind-hgt-500,cancm30,cancm40,ccsm30,ccsm40,cfsv20,gfdlflora0,gfdlflorb0,gfdl0,nasa0,nmme0mean,climateregions__climateregion,elevation__elevation,wind-vwnd-250-2010-1,wind-vwnd-250-2010-2,wind-vwnd-250-2010-3,wind-vwnd-250-2010-4,wind-vwnd-250-2010-5,wind-vwnd-250-2010-6,wind-vwnd-250-2010-7,wind-vwnd-250-2010-8,wind-vwnd-250-2010-9,wind-vwnd-250-2010-10,wind-vwnd-250-2010-11,wind-vwnd-250-2010-12,wind-vwnd-250-2010-13,wind-vwnd-250-2010-14,wind-vwnd-250-2010-15,wind-vwnd-250-2010-16,wind-vwnd-250-2010-17,wind-vwnd-250-2010-18,wind-vwnd-250-2010-19,wind-vwnd-250-2010-20,wind-uwnd-250-2010-1,wind-uwnd-250-2010-2,wind-uwnd-250-2010-3,wind-uwnd-250-2010-4,wind-uwnd-250-2010-5,wind-uwnd-250-2010-6,wind-uwnd-250-2010-7,wind-uwnd-250-2010-8,wind-uwnd-250-2010-9,wind-uwnd-250-2010-10,wind-uwnd-250-2010-11,wind-uwnd-250-2010-12,wind-uwnd-250-2010-13,wind-uwnd-250-2010-14,wind-uwnd-250-2010-15,wind-uwnd-250-2010-16,wind-uwnd-250-2010-17,wind-uwnd-250-2010-18,wind-uwnd-250-2010-19,wind-uwnd-250-2010-20,mjo1d__phase,mjo1d__amplitude,mei__mei,mei__meirank,mei__nip,wind-hgt-850-2010-1,wind-hgt-850-2010-2,wind-hgt-850-2010-3,wind-hgt-850-2010-4,wind-hgt-850-2010-5,wind-hgt-850-2010-6,wind-hgt-850-2010-7,wind-hgt-850-2010-8,wind-hgt-850-2010-9,wind-hgt-850-2010-10,sst-2010-1,sst-2010-2,sst-2010-3,sst-2010-4,sst-2010-5,sst-2010-6,sst-2010-7,sst-2010-8,sst-2010-9,sst-2010-10,wind-hgt-500-2010-1,wind-hgt-500-2010-2,wind-hgt-500-2010-3,wind-hgt-500-2010-4,wind-hgt-500-2010-5,wind-hgt-500-2010-6,wind-hgt-500-2010-7,wind-hgt-500-2010-8,wind-hgt-500-2010-9,wind-hgt-500-2010-10,icec-2010-1,icec-2010-2,icec-2010-3,icec-2010-4,icec-2010-5,icec-2010-6,icec-2010-7,icec-2010-8,icec-2010-9,icec-2010-10,wind-uwnd-925-2010-1,wind-uwnd-925-2010-2,wind-uwnd-925-2010-3,wind-uwnd-925-2010-4,wind-uwnd-925-2010

In [13]:
# Check for missing values
pd.set_option('display.max_rows', None) #display all the rows
print('missing values:', all_data.isnull().sum())

missing values: index                                           0
lat                                             0
lon                                             0
startdate                                       0
contest-pevpr-sfc-gauss-14d__pevpr              0
nmme0-tmp2m-34w__cancm30                        0
nmme0-tmp2m-34w__cancm40                        0
nmme0-tmp2m-34w__ccsm30                     15934
nmme0-tmp2m-34w__ccsm40                         0
nmme0-tmp2m-34w__cfsv20                         0
nmme0-tmp2m-34w__gfdlflora0                     0
nmme0-tmp2m-34w__gfdlflorb0                     0
nmme0-tmp2m-34w__gfdl0                          0
nmme0-tmp2m-34w__nasa0                          0
nmme0-tmp2m-34w__nmme0mean                      0
contest-wind-h10-14d__wind-hgt-10               0
nmme-tmp2m-56w__cancm3                          0
nmme-tmp2m-56w__cancm4                          0
nmme-tmp2m-56w__ccsm3                       10280
nmme-tmp2m-56w__ccsm4             

DROP NULL VALUS

In [ ]:
all_data = all_data.dropna()

In [ ]:
print('missing values:', all_data.isnull().sum())

In [14]:
 #show some information about the dataset
print(all_data.info())
print(all_data.dtypes)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375734 entries, 0 to 375733
Columns: 246 entries, index to wind-vwnd-925-2010-20
dtypes: category(2), float32(240), int32(1), int8(3)
memory usage: 347.6 MB
None
index                                          int32
lat                                          float32
lon                                          float32
startdate                                   category
contest-pevpr-sfc-gauss-14d__pevpr           float32
nmme0-tmp2m-34w__cancm30                     float32
nmme0-tmp2m-34w__cancm40                     float32
nmme0-tmp2m-34w__ccsm30                      float32
nmme0-tmp2m-34w__ccsm40                      float32
nmme0-tmp2m-34w__cfsv20                      float32
nmme0-tmp2m-34w__gfdlflora0                  float32
nmme0-tmp2m-34w__gfdlflorb0                  float32
nmme0-tmp2m-34w__gfdl0                       float32
nmme0-tmp2m-34w__nasa0                       float32
nmme0-tmp2m-34w__nmme0mean                   f

DROP COLUMNS

In [ ]:
column_headers = list(all_data.columns.values)
print("The Column Header :", column_headers)

In [ ]:

all_data = all_data.drop(['climateregions__climateregion'], axis=1)

Extract year, month and date from start data

In [26]:
all_data['startdate'] = pd.to_datetime(all_data['startdate'])
all_data['year'] = all_data['startdate'].dt.year
all_data['month'] = all_data['startdate'].dt.month
all_data['date'] = all_data['startdate'].dt.day

In [ ]:
print(all_data.dtypes)

In [35]:
#drop startdate column
all_data = all_data.drop(['startdate'], axis=1) 

In [ ]:
print(all_data.dtypes)

In [37]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 375734 entries, 2014-09-01 to 2016-08-31
Columns: 248 entries, index to date
dtypes: category(1), float32(240), int32(1), int64(3), int8(3)
memory usage: 358.3 MB


MODEL BUILDING

split data into train and test

In [ ]:
train_cleaned = all_data[:ntrain]
test_cleaned = all_data[ntrain:].reset_index(drop=True).drop('contest-tmp2m-14d__tmp2m', axis=1)

In [ ]:
train_cleaned.info()

In [ ]:
test_cleaned.info()

In [ ]:
x_train = train_cleaned.drop(['contest-tmp2m-14d__tmp2m'], axis=1) 
y_train = train_cleaned['contest-tmp2m-14d__tmp2m']

In [ ]:
print('missing values:', y_train.isnull().sum())

In [ ]:
import sklearn.model_selection
# Split train_data
from sklearn.model_selection import train_test_split

X_Train, X_Val, y_Train, y_val = train_test_split(x_train, y_train, 
                                                  test_size = 0.1, random_state=42)

In [ ]:
from sklearn import linear_model
model = linear_model.LinearRegression()

In [ ]:
x_train.dtypes

In [ ]:
print('missing values:', y_train.isnull().sum())

In [ ]:
model.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import r2_score
y_pred = model.predict(X_Val)
r2_score(y_val, y_pred)

In [ ]:
ss.head()

In [ ]:
preds = model.predict(test_cleaned)
preds = pd.DataFrame(preds, columns=['contest-tmp2m-14d__tmp2m'])
preds.head()

## SUBMISSION

In [ ]:
index = test_cleaned['index']

In [ ]:
# Create submission DataFrame
submission = pd.DataFrame(
    {
        'contest-tmp2m-14d__tmp2m': preds['contest-tmp2m-14d__tmp2m'],
        'index': index
      
    }
)

submission.head()

In [ ]:
# Create submission csv file csv file
submission.to_csv('submission.csv', index=False)